In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.optimizers.gibo import GIBOptimizer
from src.optimizers.vanilla import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "Swimmer-v4"
kernel_name = "rbfstate" ## "linearstate" /"rbfstate"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
additional_layers=[] ### can be empty or [8,7] for adding 2 layers with width 8,7 respectively


likelihood_config = {
                "noise_hyperprior":gpytorch.priors.torch_priors.UniformPrior(a=0.01,b=0.02),
                "noise_constraint":gpytorch.constraints.constraints.Interval(0.01,0.02)
                }

optimizer_config = {
        "n_eval":3, ## 3 for cartpole (very noisy)
        ### for GIBO
        "n_max":50, 
        "n_info_samples":16,
        "delta":0.1, ## 0.01 for cartpole
        ### hessian normalisation applies only for rbf
        "normalize_gradient":True if kernel_name == "rbf" else False,
        "standard_deviation_scaling":False,
}

kernel_config = {
        "use_ard":False,
        "kernel_name":kernel_name,
        # "lengthscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(3.0,6.0),
        "lengthscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.01),
        "outputscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.01),
        # "outputscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(loc=2.0,scale=1.0),
        }


trainer_config = {
        "n_steps":300, 
        "report_freq":10,
        "save_best":False,
}


model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers)


### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Using ard_num_dims = 18
 Gibo will use 50 last points to fit GP and 16 info samples


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gpytorch/lazy/lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


current 0.07676661759614944 / max 0.21548153460025787 /batch_mean 0.032693587243556976 /batch_max 0.08514556288719177 
##############################
covar_lengthscale max 77.4983901977539 / min 77.4983901977539                      covar_outputscale 1.0                     noise 0.014999999664723873
##############################
last parameters tensor([-0.1664,  0.0738,  0.0974, -0.0447,  0.1089, -0.1000,  0.1039, -0.0011,
        -0.0070,  0.0725, -0.0243, -0.0255,  0.0815, -0.0634,  0.0096, -0.0182,
        -0.1850, -0.1361])
MLL : 1.020346760749817
grad_mean : max 0.10036879777908325 /  min -0.06237058341503143
grad_covar : max 2.2576355934143066 /  min 9.999999717180685e-10
current 0.07905768603086472 / max 0.21548153460025787 /batch_mean 0.05467354506254196 /batch_max 0.07905768603086472 
##############################
covar_lengthscale max 77.4983901977539 / min 77.4983901977539                      covar_outputscale 1.0                     noise 0.014999999664723873
##########

KeyboardInterrupt: 